In [1]:
import pandas as pd
import pickle
import numpy as np
import sys
sys.path.append("../")
from src.soporte_transformers import (
    load_models
)
pd.set_option('display.max_columns',100)

In [2]:
df = pd.read_pickle("../datos/02_datos_gestionados.plk")
df.sample()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
152,34,1,Travel_Frequently,Research & Development,5 km,Bachelor,Medical,Male,Level 2,Research Scientist,Divorced,137440,None,20 %,Level 1,3,3 Times,2,1,0,Very High,High,Medium,High


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1491 entries, 0 to 1490
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   Age                      1491 non-null   int64   
 1   Attrition                1491 non-null   category
 2   BusinessTravel           1491 non-null   category
 3   Department               1491 non-null   category
 4   DistanceFromHome         1491 non-null   category
 5   Education                1491 non-null   category
 6   EducationField           1491 non-null   category
 7   Gender                   1491 non-null   category
 8   JobLevel                 1491 non-null   category
 9   JobRole                  1491 non-null   category
 10  MaritalStatus            1491 non-null   category
 11  MonthlyIncome            1491 non-null   int64   
 12  NumCompaniesWorked       1491 non-null   category
 13  PercentSalaryHike        1491 non-null   category
 14  StockOpt

# Creamos un dato nuevo
- Para probar nuestras predicciones

In [4]:
target_encoder, one_hot_encoder, robust_scaler, modelo_xgb =  load_models()

cols_target = ["BusinessTravel", "Department", "EducationField", "MaritalStatus", "NumCompaniesWorked", "EnvironmentSatisfaction", "JobSatisfaction", "WorkLifeBalance"]
cols_onehot = ["DistanceFromHome","Education", "Gender", "JobLevel", "JobRole", "PercentSalaryHike", "StockOptionLevel","TrainingTimesLastYear","JobInvolvement"]

cols_escalar = ["Age","BusinessTravel", "Department", "EducationField",
                  "MaritalStatus", "MonthlyIncome", "NumCompaniesWorked","TotalWorkingYears",
                  "YearsAtCompany","YearsSinceLastPromotion","YearsWithCurrManager",
                  "EnvironmentSatisfaction", "JobSatisfaction", "WorkLifeBalance"]


# Datos de una nueva casa para predicción
employee = pd.DataFrame({
    'Age': [13],
    'BusinessTravel': ["Travel_Rarely"],
    'Department': ["Sales"],
    'DistanceFromHome': ["11 km"],
    'Education': ["Below College"],
    'EducationField': ["Technical Degree"],
    'Gender': ["Male"],
    'JobLevel': ["Level 2"],
    'JobRole': ["Human Resources"],
    'MaritalStatus': ["Single"],
    'MonthlyIncome': [32568],
    'NumCompaniesWorked': ["3 companies"],
    'PercentSalaryHike': ["18 %"],
    'StockOptionLevel': ["Level 1"],
    'TotalWorkingYears': [12],
    'TrainingTimesLastYear': ["2 Times"],
    'YearsAtCompany': [4],
    'YearsSinceLastPromotion': [1],
    'YearsWithCurrManager': [3],
    'EnvironmentSatisfaction': ["Medium"],
    'JobSatisfaction': ["High"],
    'WorkLifeBalance': ["High"],
    'JobInvolvement' : ["High"]
})

df_pred = pd.DataFrame(employee)
df_pred

# Hacemos el OneHot Encoder
onehot = one_hot_encoder.transform(df_pred[cols_onehot])
# Obtenemos los nombres de las columnas del codificador
column_names = one_hot_encoder.get_feature_names_out(cols_onehot)
# Convertimos a un DataFrame
onehot_df = pd.DataFrame(onehot.toarray(), columns=column_names)

# Realizamos el target encoder
df_pred["Attrition"] = np.nan
df_pred = target_encoder.transform(df_pred)

# Quitamos las columnas que ya han sido onehoteadas 
df_pred.drop(columns= cols_onehot,inplace=True)
df_pred = pd.concat([df_pred, onehot_df], axis=1)

# Escalamos los valores
df_pred[cols_escalar] = robust_scaler.transform(df_pred[cols_escalar])

# Realizamos la predicción
df_pred.drop(columns="Attrition",inplace=True)
modelo_xgb.predict(df_pred)

array([0])

# Ver si puedo guardar listas en pkl


In [7]:
df = pd.read_pickle("../datos/02_datos_gestionados.plk")

In [8]:
df.sample()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
1222,33,0,Travel_Frequently,Sales,9 km,College,Marketing,Male,Level 4,Human Resources,Married,29090,2 companies,11 %,Level 0,10,2 Times,5,1,3,High,Low,High,Medium


BusinessTravel, Department, DistanceFromHome, Education, EducationField, Gender, JobLevel, JobRole, MaritalStatus, NumCompaniesWorked, PercentSalaryHike, StockOptionLevel, TrainingTimesLastYear, EnvironmentSatisfaction, JobSatisfaction, WorkLifeBalance, JobInvolvement

In [ ]:

cols_cat = df.select_dtypes("category")
cols_cat.drop(columns="Attrition",inplace=True)
for i, col in enumerate(cols_cat.columns):
    lista_opciones = cols_cat[col].unique().tolist()
    with open(f'../datos/lista_opciones/{i}_lista_opciones_{col}.plk', 'wb') as archivo:
        pickle.dump(sorted(lista_opciones), archivo)

In [17]:
df = pd.read_pickle("../datos/02_datos_gestionados.plk")
cols_cat = df.select_dtypes("category")
cols_cat.drop(columns="Attrition",inplace=True)
for i, col in enumerate(cols_cat.columns):
    with open(f'../datos/lista_opciones/{i}_lista_opciones_{col}.plk', 'rb') as archivo:
        bujia = pickle.load(archivo)
        print(bujia)

['Non-Travel', 'Travel_Frequently', 'Travel_Rarely']
['Human Resources', 'Research & Development', 'Sales']
['1 km', '10 km', '11 km', '12 km', '13 km', '14 km', '15 km', '16 km', '17 km', '18 km', '19 km', '2 km', '20 km', '21 km', '22 km', '23 km', '24 km', '25 km', '26 km', '27 km', '28 km', '29 km', '3 km', '4 km', '5 km', '6 km', '7 km', '8 km', '9 km']
['Bachelor', 'Below College', 'College', 'Doctor', 'Master']
['Human Resources', 'Life Sciences', 'Marketing', 'Medical', 'Other', 'Technical Degree']
['Female', 'Male']
['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5']
['Healthcare Representative', 'Human Resources', 'Laboratory Technician', 'Manager', 'Manufacturing Director', 'Research Director', 'Research Scientist', 'Sales Executive', 'Sales Representative']
['Divorced', 'Married', 'Single']
['1 companies', '2 companies', '3 companies', '4 companies', '5 companies', '6 companies', '7 companies', '8 companies', '9 companies', 'None']
['11 %', '12 %', '13 %', '14 %', '15 %

In [ ]:
lista_tipo_propiedad = df["propertyType"].unique().tolist()
lista_habitaciones = df["rooms"].unique().tolist()
lista_aseos = df["bathrooms"].unique().tolist()
lista_floor = df["floor"].unique().tolist()
lista_municipality = df["municipality"].unique().tolist()
lista_district = df["district"].unique().tolist()
lista_ascensor = df["hasLift"].unique().tolist()
lista_distancia = df["distancia_centro"].unique().tolist()


In [ ]:
lista_distancia = ['Menos de 1 km', 'Entre 1 y 5 km', 'Entre 5 y 10 km', 'Entre 10 y 20 km', 'Entre 20 y 30 km', 'Entre 30 y 40 km', 'Entre 40 y 50 km', 'Mas de 50 km']

In [ ]:
lista_floor = ['sotano',"bajo","entreplanta",'primero',"segundo","tercero","cuarto","quinto","sexto","septimo","octavo","decimo cuarto","desconocido"]

### Guardarlos en un pickle

In [ ]:
with open('../datos/lista_opciones/01_tipo_propiedad.pkl', 'wb') as archivo:
    pickle.dump(sorted(lista_tipo_propiedad), archivo)
with open('../datos/lista_opciones/02_num_habitaciones.pkl', 'wb') as archivo:
    pickle.dump(sorted(lista_habitaciones), archivo)
with open('../datos/lista_opciones/03_num_aseos.pkl', 'wb') as archivo:
    pickle.dump(sorted(lista_aseos), archivo)
with open('../datos/lista_opciones/04_floor.pkl', 'wb') as archivo:
    pickle.dump(lista_floor, archivo)
with open('../datos/lista_opciones/05_municipio.pkl', 'wb') as archivo:
    pickle.dump(sorted(lista_municipality), archivo)
with open('../datos/lista_opciones/06_distrito.pkl', 'wb') as archivo:
    pickle.dump(sorted(lista_district), archivo)
with open('../datos/lista_opciones/07_ascensor.pkl', 'wb') as archivo:
    pickle.dump(sorted(lista_ascensor), archivo)
with open('../datos/lista_opciones/08_distancia_centro.pkl', 'wb') as archivo:
    pickle.dump(lista_distancia, archivo)

In [ ]:
with open('../datos/lista_opciones/01_tipo_propiedad.pkl', 'rb') as archivo:
    tipo_propiedad = pickle.load(archivo)

In [ ]:
tipo_propiedad

['chalet', 'countryHouse', 'duplex', 'flat', 'penthouse', 'studio']